<a href="https://colab.research.google.com/github/vinejain/Intel-Image-Classification-Task/blob/TestingTFV2/intel_image_classification_VJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Changing tfv2 to tfv1 on Colab didn't work due to an on-going issue (https://github.com/tensorflow/tensorflow/issues/30531)

Created a new git branch and trying back with tfv2 on Colab

Checking TF version in colab

In [1]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
#!pip install -q kaggle
!pip install kaggle

# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle (3).json


In [2]:
!mkdir -p ~/.kaggle/
!cp kaggle.json ~/.kaggle/
#change the permission
!chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d puneet6060/intel-image-classification

intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!ls

 intel-image-classification.zip  'kaggle (3).json'   seg_pred
'kaggle (1).json'		  kaggle.json	     seg_test
'kaggle (2).json'		  sample_data	     seg_train


In [4]:
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from keras.utils.vis_utils import model_to_dot
import os
import matplotlib.pyplot as plot
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix as CM
from random import randint
from IPython.display import SVG
import matplotlib.gridspec as gridspec

Using TensorFlow backend.


In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


Upgrading runtime type to TPU

In [5]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  #with tf.Session(tpu_address) as session:
  #https://stackoverflow.com/questions/55142951/tensorflow-2-0-attributeerror-module-tensorflow-has-no-attribute-session
  with tf.compat.v1.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.114.132.90:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 201674499683107317),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, -6467609320016107918),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9043468490058826835),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4079219975035746585),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, -2640054822101537736),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 8447319959128945062),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 161675643477532916),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 722797569586095262),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 5945781267744895

In [8]:
#tf.logging.set_verbosity(tf.logging.DEBUG)

# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

'''
resnet_model = tf.contrib.tpu.keras_to_tpu_model(
    resnet_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))
'''

'\nresnet_model = tf.contrib.tpu.keras_to_tpu_model(\n    resnet_model,\n    strategy=tf.contrib.tpu.TPUDistributionStrategy(\n        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))\n'

In [9]:
from zipfile import ZipFile
file_name = "intel-image-classification.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")

Done


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
directory = '/content/seg_train/seg_train'
def get_images(directory):
    Images = []
    Labels = []  # 0 for Building , 1 for forest, 2 for glacier, 3 for mountain, 4 for Sea , 5 for Street
    label = 0
    
    for labels in os.listdir(directory): #Main Directory where each class label is present as folder name.
        if labels == 'glacier': #Folder contain Glacier Images get the '2' class label.
            label = 2
        elif labels == 'sea':
            label = 4
        elif labels == 'buildings':
            label = 0
        elif labels == 'forest':
            label = 1
        elif labels == 'street':
            label = 5
        elif labels == 'mountain':
            label = 3
        
        for image_file in os.listdir(directory+labels): #Extracting the file name of the image from Class Label folder
            image = cv2.imread(directory+labels+r'/'+image_file) #Reading the image (OpenCV)
            image = cv2.resize(image,(150,150)) #Resize the image, Some images are different sizes. (Resizing is very Important)
            Images.append(image)
            Labels.append(label)
    
    return shuffle(Images,Labels,random_state=817328462) #Shuffle the dataset you just prepared.

def get_classlabel(class_code):
    labels = {2:'glacier', 4:'sea', 0:'buildings', 1:'forest', 5:'street', 3:'mountain'}
    
    return labels[class_code]

In [7]:
directory = '/content/seg_train/seg_train/'

#Images, Labels = get_images('../input/seg_train/seg_train/') #Extract the training images from the folders.
Images, Labels = get_images(directory) #Extract the training images from the folders.

Images = np.array(Images) #converting the list of images to numpy array.
Labels = np.array(Labels)

In [8]:
print("Shape of Images:",Images.shape)
print("Shape of Labels:",Labels.shape)

Shape of Images: (14034, 150, 150, 3)
Shape of Labels: (14034,)


**Let us look some random images of our dataset.**

In [ ]:
f,ax = plot.subplots(5,5) 
f.subplots_adjust(0,0,3,3)
for i in range(0,5,1):
    for j in range(0,5,1):
        rnd_number = randint(0,len(Images))
        ax[i,j].imshow(Images[rnd_number])
        ax[i,j].set_title(get_classlabel(Labels[rnd_number]))
        ax[i,j].axis('off')

**Now, Create the CNN model to predict the class labels.This is the basic CNN model.**

tf.contrib has moved out of TF starting TF 2.0 alpha.
Take a look at these tf 2.0 release notes https://github.com/tensorflow/tensorflow/releases/tag/v2.0.0-alpha0
You can upgrade your TF 1.x code to TF 2.x using the tf_upgrade_v2 script https://www.tensorflow.org/alpha/guide/upgrade

In [9]:
tf.config.experimental_connect_to_host('grpc://' + os.environ['COLAB_TPU_ADDR'])
resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.114.132.90:8470
INFO:tensorflow:Clearing out eager caches
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/upgrade.ipynb#scrollTo=se9Leqjm1CZR

https://stackoverflow.com/questions/54521572/how-to-transform-keras-model-to-tpu-model

https://stackoverflow.com/questions/55324762/the-added-layer-must-be-an-instance-of-class-layer-found-tensorflow-python-ke

https://github.com/tensorflow/tensorflow/issues/37426

https://medium.com/@jannik.zuern/using-a-tpu-in-google-colab-54257328d7da

https://colab.research.google.com/drive/1XvhLFD4zECIY6wLQJHDgZORVGGY2GDJa#scrollTo=z0DRPKl39290

https://colab.research.google.com/drive/1ISfhxFDntfOos7cOeT7swduSqzLEqyFn#scrollTo=VQWJ-urr99OC

https://stackoverflow.com/questions/55870127/module-tensorflow-has-no-attribute-contrib

https://stackoverflow.com/questions/54953008/tensorflow-error-while-loading-pre-trained-resnet-model

In [10]:
from keras.layers import Activation, Dense

with strategy.scope():
  model = Models.Sequential()
  model.add(Layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
  model.add(Layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
  model.add(Layers.MaxPool2D(5,5))
  model.add(Layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
  model.add(Layers.Conv2D(140,kernel_size=(3,3),activation='relu'))
  model.add(Layers.Conv2D(100,kernel_size=(3,3),activation='relu'))
  model.add(Layers.Conv2D(50,kernel_size=(3,3),activation='relu'))
  model.add(Layers.MaxPool2D(5,5))
  model.add(Layers.Flatten())
  model.add(Layers.Dense(180,activation='relu'))
  model.add(Layers.Dense(100,activation='relu'))
  model.add(Layers.Dense(50,activation='relu'))
  model.add(Layers.Dropout(rate=0.5))
  model.add(Layers.Dense(6,activation='softmax'))
  model.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])



In [17]:
with strategy.scope():
  model = tf.keras.Sequential()
  model.add(Dense(32, input_dim=784))
  model.add(Dense(32))
  model.add(Activation('relu'))
  model.compile(optimizer='rmsprop', loss='mse')

TypeError: ignored

**Now, lets train the model with validation split equal to 30% from training data. Validation split is used to determine that our model is not geting over-fitted.**

In [12]:
trained = model.fit(Images,Labels,epochs=35,validation_split=0.30)

NotFoundError: ignored

In [ ]:
plot.plot(trained.history['acc'])
plot.plot(trained.history['val_acc'])
plot.title('Model accuracy')
plot.ylabel('Accuracy')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

plot.plot(trained.history['loss'])
plot.plot(trained.history['val_loss'])
plot.title('Model loss')
plot.ylabel('Loss')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

NameError: ignored